# Recomendaciones

***1. Se tiene la siguiente matriz que relaciona a usuarios (A, B, C y D) y productos comprados (P1, P2, etc.)***

|    | P1 | P2 | P3 | P4 | P5 | P6 | P7 | P8 |
|----|----|----|----|----|----|----|----|----|
| A  |    | 1  |    |    | 1  | 1  |    |    |
| B  |    |    | 1  |    |    | 1  | 1  | 1  |
| C  | 1  |    |    |    | 1  | 1  |    |    |
| D  |    | 1  |    | 1  |    |    | 1  |    |

***Usando la semejanza de Jaccard y Collaborative Filtering user-user con k = 2, indicar cuáles serían los 3 productos que le recomendaría al usuario A y en qué orden. En caso de que haya empates, usar
Collaborative Filtering item-item para desempatar.***

Como la matriz indica la cantidad de productos comprados porr usuario, tiene sentido usar la semejanza de Jaccard. Por lo tanto, como tenemos que usar CF User-User, vamos a ver cuales son los k=2 usuarios más semejantes al usuario A.

In [6]:
jaccard = lambda x,y: len(x & y)/len(x | y)

In [23]:
A = {'p2', 'p5', 'p6'}
B = {'p3', 'p6', 'p7', 'p8'}
C = {'p1', 'p5', 'p6'}
D = {'p2', 'p4', 'p7'}

for userdata in [(A,'A'),(B,'B'),(C,'C'),(D,'D')]:
    user = userdata[0]
    username = userdata[1]
    print(f"Sim(A,{username}) = {jaccard(A,user)}")

Sim(A,A) = 1.0
Sim(A,B) = 0.16666666666666666
Sim(A,C) = 0.5
Sim(A,D) = 0.2


|    | P1 | P2 | P3 | P4 | P5 | P6 | P7 | P8 |
|----|----|----|----|----|----|----|----|----|
| A  |    | 1  |    |    | 1  | 1  |    |    |
| B  |    |    | 1  |    |    | 1  | 1  | 1  |
| C  | 1  |    |    |    | 1  | 1  |    |    |
| D  |    | 1  |    | 1  |    |    | 1  |    |

Los dos usuarios más semejantes a A son el C y D, por lo tanto debo comparar con ellos. Ahora sé que mis raitings faltantes los puedo calcular en función de los raitings de C y D. Asumo que los espacios faltantes son productos no comprados, pero en el resto de ejercicios no se puede.

```
R_a1 = (0.5 * 1 + 0 * 0.2)/0.7 = 1
R_a3 = 0
R_a4 = (0.5 * 0 + 1 * 0.2)/0.7 = 0.2857
R_a7 = (0.5 * 0 + 1 * 0.2)/0.7 = 0.2857
R_a8 = 0
``` 

Hasta ahora recomendaríamos el producto 1 en primer lugar y faltaría desempatar por los productos 4 y 7 usando CF ítem-ítem.

In [24]:
P2 = {'A', 'D'}
P4 = {'D'}
P7 = {'B', 'D'}
P5 = {'A', 'C'}
P6 = {'A', 'B', 'C'}

for productdata in [(P2, 'P2'), (P5, 'P5'), (P6, 'P6')]:
    product = productdata[0]
    productname = productdata[1]
    print(f"Sim(P4,{productname}) = {jaccard(P4,product)}")     
    print(f"Sim(P7,{productname}) = {jaccard(P7,product)}") 

Sim(P4,P2) = 0.5
Sim(P7,P2) = 0.3333333333333333
Sim(P4,P5) = 0.0
Sim(P7,P5) = 0.0
Sim(P4,P6) = 0.0
Sim(P7,P6) = 0.25


Vamos a desempatar con esta fórmula:

$R_{User, Ítem} = \frac{\sum \text{Sim}(Ítem,x) \cdot R_{User,x}}{\sum \text{Sim}(Ítem,x)}$ siendo x el resto de ítems.

```
R_a4 = (1/2)/(1/2+1/2) = 1/2
R_a7 = (1/3+1/4)/(1/3+1/4+3*1/2) = 0.28
```

Por lo tanto el orden final de recomendación es: $R_{a,p_1} > R_{a,p_4} > R_{a,p_7}$

***2. a. Usando Collaborative Filtering item-item calcular la calificación del usuario 2 para la película 1, considerando 2 items similares.***

***2. b. Usando Collaborative Filtering en base a desviaciones calcular la calificación del usuario 2 para la película
1, considerando 2 items similares.***

***2. c. ¿Qué conclusiones se pueden sacar de la comparación de ambos resultados?***


Debo calcular que dos ítems son más similares entre la película 1 y el resto.

```
Sim(p1,p2) = -0.9609069544270727
Sim(p1,p3) =  0.6575959492214292
Sim(p1,p4) = -0.837610596838614
Sim(p1,p5) = -0.8890008890013337
Sim(p1,p6) =  0.7675194639536289
```

Las películas más similares a la 1 son la 3 y 6. Ahora debo calcular el resultado esperado:

```
R_user2,p1 = R_p1,user2 = (Sim(p1,p3) * 4)/Sim(p1,p3) = 4
```

No usamos la película 6 pues no la vio el usuario

Ahora debo calcular en función de las desviaciones: $\mu = 3.17, \delta_{U2} = -0.17, \delta_{P1} = 0.43$

```
delta_{U2,P1} = (0.6576 * (4 - (3.17 - 0.17 + 0.43 - 0.17)))/0.6576 = 4 - (3.17 - 0.17 + 0.43 - 0.17) = 0.74
```

(siendo que $\delta_{P3} = 3 - \mu = -0.17$)

El raiting final nos queda: 

``` 
3.17 - 0.17 + 0.43 + 0.74 = 4.17
``` 

***8. Se sabe que el promedio global de un conjunto de calificaciones de películas es de 2,28. Sabiendo que el usuario “Ariel” ha realizado las siguientes calificaciones: 1, 4, 2, 1, 1; y que la película “Interstellar” tiene las siguientes calificaciones: 5, 4, 1, 3; ¿cuál sería la estimación de la calificación de “Ariel” para “Interstellar”?***



Sabemos que $\mu = 2.28$. Luego $\delta_{\text{Ariel}} = \text{avg}(\text{Ariel}) - \mu = \frac{1 + 4 + 2 + 1 + 1}{5} - 2.28 = -0.48$ y $\delta_{\text{Interstellar}} = \text{avg}(\text{Interstellar}) - \mu = \frac{5 + 4 + 1 + 3}{4} - 2.28 = 0.97$.

Falta calcular $\delta_{\text{Ariel, Interstellar}}$ pero sin data de otras películas solo puedo asumir que $\delta_{\text{Ariel, Interstellar}} = 0 \implies R_{\text{Ariel, Interstellar}} = \mu + \delta_{\text{Ariel}} + \delta_{\text{Interstellar}} = 2.28 - 0.48 + 0.97 = 2.77$